In [77]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime, timedelta
import numpy as np

## Analisando dados de teste

In [78]:
df = pd.read_csv('idsc_test.csv')

In [79]:
# Pegando os pares numericos do snapshot_radar
rota = df['snapshot_radar'].head(1).values[0]
coordenadas = re.findall(r'-?\d+\.\d+', rota)
pares = [(float(coordenadas[i]), float(coordenadas[i+1])) for i in range(0, len(coordenadas), 2)]
map = pd.DataFrame(pares, columns=['Latitude', 'Longitude'])

In [80]:
df.head(1).T

,0
flightid,7945735584a3297121c4f5ae0de8ecd1
origem,SBKP
destino,SBSV
dt_dep,2023-05-30 11:36:03.000
hora_ref,2023-05-30 11:00:00.000
snapshot_radar,MULTIPOINT ((-0.8807200448127578 -0.5148453155...
path,http://satelite.cptec.inpe.br/repositoriogoes/...
hora_esperas,2023-05-30 10:00:00.000
esperas,0
aero_esperas,SBSV


In [81]:
colunas = ['dt_dep', 'hora_ref', 'hora_esperas', 'hora_metar', 'hora_metaf', 'hora_tcp']

# 1. Converter as colunas para datetime de uma só vez
df[colunas] = df[colunas].apply(pd.to_datetime)

# 2. Encontrar índices de linhas com 'hora_metaf' nula
null_indices = df.index[df['hora_metaf'].isna()]

# 3. Atualizar as colunas com base nas colunas 'hora_metar' e 'metar'
df.loc[null_indices, 'hora_metaf'] = df.loc[null_indices, 'hora_metar'] + timedelta(hours=1)
df.loc[null_indices, 'metaf'] = df.loc[null_indices, 'metar']
df.loc[null_indices, 'aero_metaf'] = df.loc[null_indices, 'aero_metar']

# 4. Converter as colunas para valores inteiros UNIX timestamp
# df[colunas] = df[colunas].apply(lambda x: x.dt.timestamp()).astype(int)

## Analisando dados da API

In [82]:
def read_and_process_csv(file_path, date_columns=None, rename_columns=None):
    df = pd.read_csv(file_path)
    print(df.info())
    
    if date_columns:
        for col in date_columns:
            df[col] = pd.to_datetime(df[col].apply(convert), format='%d/%m/%Y %H:%M:%S')
    if rename_columns:
        df = df.rename(columns=rename_columns)
    
    return df

# Define a função de conversão
def convert(timestamp):
    data_hora = datetime.fromtimestamp(float(timestamp)/1000.0)
    return data_hora.strftime('%d/%m/%Y %H:%M:%S')

### Bimtra

In [83]:
bimtra = read_and_process_csv('Dados/bimtra.csv', date_columns=['dt_dep', 'dt_arr'], rename_columns={'hora': 'hora_ref'})
bimtra['hora_ref'] = bimtra['dt_dep'].dt.floor('H')
bimtra

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300347 entries, 0 to 300346
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   flightid  300347 non-null  object
 1   origem    300347 non-null  object
 2   destino   300347 non-null  object
 3   dt_dep    300347 non-null  int64 
 4   dt_arr    300347 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB
None


,flightid,origem,destino,dt_dep,dt_arr,hora_ref
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-05-31 21:44:57,2022-05-31 22:26:00,2022-05-31 21:00:00
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-05-31 21:57:01,2022-05-31 22:32:53,2022-05-31 21:00:00
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-05-31 21:26:52,2022-05-31 22:43:45,2022-05-31 21:00:00
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-05-31 21:58:40,2022-05-31 22:41:41,2022-05-31 21:00:00
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-05-31 22:00:38,2022-05-31 23:33:05,2022-05-31 22:00:00
...,...,...,...,...,...,...
300342,c4321a6a09b76969df53965fb5390f12,SBBR,SBRF,2023-05-13 09:14:19,2023-05-13 11:20:23,2023-05-13 09:00:00
300343,e4f5686cce45083566945c8946e5d120,SBBR,SBGR,2023-05-13 06:33:15,2023-05-13 07:58:41,2023-05-13 06:00:00
300344,04f65a7c44cf176215520b4c9b4eb37f,SBCF,SBRF,2023-05-13 13:53:53,2023-05-13 16:00:24,2023-05-13 13:00:00
300345,f72a0718e581d01c695faeeff35d81f0,SBSV,SBGR,2023-05-13 02:28:58,2023-05-13 04:54:41,2023-05-13 02:00:00


### Esperas

In [84]:
esperas = read_and_process_csv('Dados/esperas.csv', date_columns=['hora'], rename_columns={'hora': 'hora_esperas', 'aero': 'aero_esperas'})
esperas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99936 entries, 0 to 99935
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   esperas  99936 non-null  int64 
 1   hora     99936 non-null  int64 
 2   aero     99936 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB
None


,esperas,hora_esperas,aero_esperas
0,0,2022-05-31 21:00:00,SBBR
1,0,2022-05-31 22:00:00,SBBR
2,0,2022-05-31 23:00:00,SBBR
3,0,2022-06-01 00:00:00,SBBR
4,0,2022-06-01 01:00:00,SBBR
...,...,...,...
99931,0,2023-05-13 16:00:00,SBSV
99932,0,2023-05-13 17:00:00,SBSV
99933,0,2023-05-13 18:00:00,SBSV
99934,0,2023-05-13 19:00:00,SBSV


In [85]:
esperas['hora_esperas'] = esperas['hora_esperas'] + timedelta(hours=1)
esperas['hora_esperas']

0       2022-05-31 22:00:00
1       2022-05-31 23:00:00
2       2022-06-01 00:00:00
3       2022-06-01 01:00:00
4       2022-06-01 02:00:00
                ...        
99931   2023-05-13 17:00:00
99932   2023-05-13 18:00:00
99933   2023-05-13 19:00:00
99934   2023-05-13 20:00:00
99935   2023-05-13 21:00:00
Name: hora_esperas, Length: 99936, dtype: datetime64[ns]

In [86]:
merge1 = pd.merge(bimtra, esperas, left_on=['hora_ref', 'destino'], right_on=['hora_esperas', 'aero_esperas'], how='inner')

In [87]:
merge1['hora_esperas']  = merge1['hora_esperas'] - timedelta(hours=1)
merge1

,flightid,origem,destino,dt_dep,dt_arr,hora_ref,esperas,hora_esperas,aero_esperas
0,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-05-31 22:00:38,2022-05-31 23:33:05,2022-05-31 22:00:00,0,2022-05-31 21:00:00,SBFL
1,b0fd0f83644625ecc21f5261e8e5e347,SBPA,SBFL,2022-05-31 22:02:51,2022-05-31 22:44:53,2022-05-31 22:00:00,0,2022-05-31 21:00:00,SBFL
2,4a161657834f9c0d4ba4ff73f4d90500,SBGR,SBFL,2022-05-31 22:53:42,2022-05-31 23:47:28,2022-05-31 22:00:00,0,2022-05-31 21:00:00,SBFL
3,052050016538ab736a254313ef009a6b,SBRJ,SBCF,2022-06-01 06:20:04,2022-06-01 07:10:10,2022-06-01 06:00:00,0,2022-06-01 05:00:00,SBCF
4,9d609ef9fcab19b182fc893eec0f0146,SBBR,SBCF,2022-06-01 06:08:39,2022-06-01 07:02:31,2022-06-01 06:00:00,0,2022-06-01 05:00:00,SBCF
...,...,...,...,...,...,...,...,...,...
300295,b7b368e8a21a45f1e3f3c2dc6f6a3951,SBGR,SBRF,2023-05-13 12:49:55,2023-05-13 15:25:40,2023-05-13 12:00:00,0,2023-05-13 11:00:00,SBRF
300296,f4c17fb0a3e3b37033758ffd83c32776,SBRF,SBSV,2023-05-13 18:22:34,2023-05-13 20:05:03,2023-05-13 18:00:00,0,2023-05-13 17:00:00,SBSV
300297,86bd325bc721c125f26191afe7eeab19,SBRJ,SBSV,2023-05-13 18:58:59,2023-05-13 20:40:36,2023-05-13 18:00:00,0,2023-05-13 17:00:00,SBSV
300298,84eed97014302c71ca93ea8fec31d6dd,SBBR,SBRF,2023-05-12 21:46:35,2023-05-12 23:56:14,2023-05-12 21:00:00,0,2023-05-12 20:00:00,SBRF


### METAF

In [88]:
metaf = read_and_process_csv('Dados/metaf.csv', date_columns=['hora'], rename_columns={'hora': 'hora_metaf', 'aero': 'aero_metaf'})
metaf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43938 entries, 0 to 43937
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hora    43938 non-null  int64 
 1   metaf   43938 non-null  object
 2   aero    43938 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


,hora_metaf,metaf,aero_metaf
0,2022-08-04 21:00:00,METAF SBBR 050000Z 05006KT CAVOK 18/04 Q1014=\n,SBBR
1,2022-08-04 22:00:00,METAF SBBR 050100Z 01006KT CAVOK 17/05 Q1016=\n,SBBR
2,2022-08-04 23:00:00,METAF SBBR 050200Z 35008KT CAVOK 17/05 Q1014=\n,SBBR
3,2022-08-05 00:00:00,METAF SBBR 050300Z 34007KT CAVOK 16/05 Q1015=\n,SBBR
4,2022-08-05 01:00:00,METAF SBBR 050400Z 34008KT CAVOK 16/05 Q1015=\n,SBBR
...,...,...,...
43933,2023-05-13 16:00:00,METAF SBSP 131900Z 14010KT 9999 OVC024 ...,SBSP
43934,2023-05-13 17:00:00,METAF SBSP 132000Z 14009KT 9999 BKN033 ...,SBSP
43935,2023-05-13 18:00:00,METAF SBSP 132100Z 14008KT 8000 OVC033 ...,SBSP
43936,2023-05-13 19:00:00,METAF SBSP 132200Z 14007KT 7000 OVC033 ...,SBSP


In [89]:
metaf['hora_metaf'] = metaf['hora_metaf'] - timedelta(hours=1)
merge1 = pd.merge(merge1, metaf, left_on=['hora_ref', 'destino'], right_on=['hora_metaf', 'aero_metaf'], how='inner')
merge1['hora_metaf']  = merge1['hora_metaf'] + timedelta(hours=1)
merge1

,flightid,origem,destino,dt_dep,dt_arr,hora_ref,esperas,hora_esperas,aero_esperas,hora_metaf,metaf,aero_metaf
0,2127312fea57d093815e49b5aefee46d,SBBR,SBGL,2022-06-01 08:08:46,2022-06-01 09:27:43,2022-06-01 08:00:00,0,2022-06-01 07:00:00,SBGL,2022-06-01 09:00:00,METAF SBGL 011200Z 15002KT 2000 BR BKN033...,SBGL
1,e20845be25ed24b87131066d865cd9bc,SBFL,SBSP,2022-06-01 09:12:29,2022-06-01 10:01:07,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP
2,79ffad2c4f0b8f0d6a25977140daf9a9,SBCT,SBSP,2022-06-01 09:01:48,2022-06-01 09:47:00,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP
3,2b846c207720b4eb92a3da158ac6cf68,SBCT,SBSP,2022-06-01 09:39:03,2022-06-01 10:24:51,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP
4,f51e68eeabfa16c625e7454a8974333e,SBRJ,SBSP,2022-06-01 09:46:40,2022-06-01 10:31:18,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP
...,...,...,...,...,...,...,...,...,...,...,...,...
152736,9ff5febb9d79c517679bf20050066027,SBPA,SBKP,2023-05-13 10:55:01,2023-05-13 12:16:19,2023-05-13 10:00:00,0,2023-05-13 09:00:00,SBKP,2023-05-13 11:00:00,METAF SBKP 131400Z 12006KT CAVOK 19/10 Q1020=\n,SBKP
152737,49681283735fdda95885adaebdad245e,SBRJ,SBBR,2023-05-13 13:15:41,2023-05-13 14:42:23,2023-05-13 13:00:00,0,2023-05-13 12:00:00,SBBR,2023-05-13 14:00:00,METAF SBBR 131700Z 18003KT CAVOK 27/11 Q1014=\n,SBBR
152738,0ea8f882431d6fbe1f431a0593bbf54d,SBKP,SBBR,2023-05-13 13:52:11,2023-05-13 15:07:12,2023-05-13 13:00:00,0,2023-05-13 12:00:00,SBBR,2023-05-13 14:00:00,METAF SBBR 131700Z 18003KT CAVOK 27/11 Q1014=\n,SBBR
152739,b4251c41bdc3e827ec18c89a40bbd45c,SBGR,SBGL,2023-05-13 17:11:29,2023-05-13 17:48:38,2023-05-13 17:00:00,0,2023-05-13 16:00:00,SBGL,2023-05-13 18:00:00,METAF SBGL 132100Z 16006KT 9999 OVC033 ...,SBGL


### METAR

In [90]:
metar = read_and_process_csv('Dados/metar.csv', date_columns=['hora'], rename_columns={'hora': 'hora_metar', 'aero': 'aero_metar'})
metar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106744 entries, 0 to 106743
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hora    106744 non-null  int64 
 1   metar   106744 non-null  object
 2   aero    106744 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.4+ MB
None


,hora_metar,metar,aero_metar
0,2022-05-31 21:00:00,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,2022-05-31 22:00:00,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,2022-05-31 23:00:00,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,2022-06-01 00:00:00,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,2022-06-01 01:00:00,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
...,...,...,...
106739,2023-05-13 16:00:00,METAR SBSV 131900Z 16009KT 9999 BKN020 FEW021T...,SBSV
106740,2023-05-13 17:00:00,METAR SBSV 132000Z 13007KT 9999 BKN020 29/24 Q...,SBSV
106741,2023-05-13 18:00:00,METAR SBSV 132100Z 12005KT 9999 SCT020 FEW025T...,SBSV
106742,2023-05-13 19:00:00,METAR SBSV 132200Z 11006KT 9999 SCT020 FEW025T...,SBSV


In [91]:
merge1 = pd.merge(merge1, metar, left_on=['hora_ref', 'destino'], right_on=['hora_metar', 'aero_metar'], how='inner')
merge1

,flightid,origem,destino,dt_dep,dt_arr,hora_ref,esperas,hora_esperas,aero_esperas,hora_metaf,metaf,aero_metaf,hora_metar,metar,aero_metar
0,2127312fea57d093815e49b5aefee46d,SBBR,SBGL,2022-06-01 08:08:46,2022-06-01 09:27:43,2022-06-01 08:00:00,0,2022-06-01 07:00:00,SBGL,2022-06-01 09:00:00,METAF SBGL 011200Z 15002KT 2000 BR BKN033...,SBGL,2022-06-01 08:00:00,METAR SBGL 011100Z 10003KT 070V130 7000 FEW009...,SBGL
1,e20845be25ed24b87131066d865cd9bc,SBFL,SBSP,2022-06-01 09:12:29,2022-06-01 10:01:07,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP,2022-06-01 09:00:00,METAR SBSP 011200Z 08003KT 030V130 9999 -RA SC...,SBSP
2,79ffad2c4f0b8f0d6a25977140daf9a9,SBCT,SBSP,2022-06-01 09:01:48,2022-06-01 09:47:00,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP,2022-06-01 09:00:00,METAR SBSP 011200Z 08003KT 030V130 9999 -RA SC...,SBSP
3,2b846c207720b4eb92a3da158ac6cf68,SBCT,SBSP,2022-06-01 09:39:03,2022-06-01 10:24:51,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP,2022-06-01 09:00:00,METAR SBSP 011200Z 08003KT 030V130 9999 -RA SC...,SBSP
4,f51e68eeabfa16c625e7454a8974333e,SBRJ,SBSP,2022-06-01 09:46:40,2022-06-01 10:31:18,2022-06-01 09:00:00,0,2022-06-01 08:00:00,SBSP,2022-06-01 10:00:00,METAF SBSP 011300Z 07004KT 4000 BR OVC033 ...,SBSP,2022-06-01 09:00:00,METAR SBSP 011200Z 08003KT 030V130 9999 -RA SC...,SBSP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152430,9ff5febb9d79c517679bf20050066027,SBPA,SBKP,2023-05-13 10:55:01,2023-05-13 12:16:19,2023-05-13 10:00:00,0,2023-05-13 09:00:00,SBKP,2023-05-13 11:00:00,METAF SBKP 131400Z 12006KT CAVOK 19/10 Q1020=\n,SBKP,2023-05-13 10:00:00,METAR SBKP 131300Z 13011KT CAVOK 18/10 Q1021=,SBKP
152431,49681283735fdda95885adaebdad245e,SBRJ,SBBR,2023-05-13 13:15:41,2023-05-13 14:42:23,2023-05-13 13:00:00,0,2023-05-13 12:00:00,SBBR,2023-05-13 14:00:00,METAF SBBR 131700Z 18003KT CAVOK 27/11 Q1014=\n,SBBR,2023-05-13 13:00:00,METAR SBBR 131600Z 14003KT 9999 FEW040 26/14 Q...,SBBR
152432,0ea8f882431d6fbe1f431a0593bbf54d,SBKP,SBBR,2023-05-13 13:52:11,2023-05-13 15:07:12,2023-05-13 13:00:00,0,2023-05-13 12:00:00,SBBR,2023-05-13 14:00:00,METAF SBBR 131700Z 18003KT CAVOK 27/11 Q1014=\n,SBBR,2023-05-13 13:00:00,METAR SBBR 131600Z 14003KT 9999 FEW040 26/14 Q...,SBBR
152433,b4251c41bdc3e827ec18c89a40bbd45c,SBGR,SBGL,2023-05-13 17:11:29,2023-05-13 17:48:38,2023-05-13 17:00:00,0,2023-05-13 16:00:00,SBGL,2023-05-13 18:00:00,METAF SBGL 132100Z 16006KT 9999 OVC033 ...,SBGL,2023-05-13 17:00:00,METAR SBGL 132000Z 14008KT 9999 SCT023 BKN040 ...,SBGL


### Satalite

In [92]:
satelite=pd.read_csv('Dados/satelite.csv')
satelite

,data,path,tamanho
0,2022-06-01 01:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1879673
1,2022-06-01 02:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1877693
2,2022-06-01 03:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1887299
3,2022-06-01 04:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1890950
4,2022-06-01 05:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1892776
...,...,...,...
8282,2023-05-13 19:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1830030
8283,2023-05-13 20:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1828890
8284,2023-05-13 21:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1823160
8285,2023-05-13 22:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1809551


### TCP

In [93]:
tcp = read_and_process_csv('Dados/tc-prev.csv', date_columns=['hora'], rename_columns={'hora': 'hora_tcp', 'aero': 'aero_tcp'})
tcp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99936 entries, 0 to 99935
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hora    99936 non-null  int64 
 1   troca   99936 non-null  int64 
 2   aero    99936 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB
None


,hora_tcp,troca,aero_tcp
0,2022-05-31 21:00:00,0,BR
1,2022-05-31 22:00:00,0,BR
2,2022-05-31 23:00:00,0,BR
3,2022-06-01 00:00:00,0,BR
4,2022-06-01 01:00:00,0,BR
...,...,...,...
99931,2023-05-13 16:00:00,0,SV
99932,2023-05-13 17:00:00,0,SV
99933,2023-05-13 18:00:00,0,SV
99934,2023-05-13 19:00:00,0,SV


In [94]:
def idf(row):
     return 'SB'+ row.loc['aero_tcp']

tcp['aero_tcp'] = tcp.apply(idf, axis=1)
tcp

,hora_tcp,troca,aero_tcp
0,2022-05-31 21:00:00,0,SBBR
1,2022-05-31 22:00:00,0,SBBR
2,2022-05-31 23:00:00,0,SBBR
3,2022-06-01 00:00:00,0,SBBR
4,2022-06-01 01:00:00,0,SBBR
...,...,...,...
99931,2023-05-13 16:00:00,0,SBSV
99932,2023-05-13 17:00:00,0,SBSV
99933,2023-05-13 18:00:00,0,SBSV
99934,2023-05-13 19:00:00,0,SBSV


In [95]:
tcp['hora_tcp'] = tcp['hora_tcp'] - timedelta(hours=1)
merge1 = pd.merge(merge1, tcp, left_on=['hora_ref', 'destino'], right_on=['hora_tcp', 'aero_tcp'], how='inner')
merge1['hora_tcp']  = merge1['hora_tcp'] + timedelta(hours=1)

### TCR

In [287]:
tcr = read_and_process_csv('Dados/tc-real.csv', date_columns=['hora'], rename_columns={'hora': 'hora_tcr', 'aero': 'aero_tcr'})
tcr

,hora_tcr,nova_cabeceira,antiga_cabeceira,aero_tcr
0,2022-06-01 11:14:03,32,03,FL
1,2022-06-01 15:51:10,18,12,RF
2,2022-06-01 16:12:25,16,13,CF
3,2022-06-01 16:12:38,15,26,GL
4,2022-06-01 16:12:57,15,26,GL
...,...,...,...,...
33426,2023-05-13 12:32:28,14,08,FL
33427,2023-05-13 19:06:36,17R,12,SP
33428,2023-05-13 12:41:44,15,26,CT
33429,2023-05-13 19:41:08,17R,12,SP


## Junção

Considerando a referência da coluna (`hora_ref`):

- `hora_espera` é ajustada para -1 hora.
- `hora_metaf` é ajustada para +1 hora.
- `hora_metar` permanece inalterada.
- `hora_tcp` é ajustada para +1 hora.
- `hora_tcr` permanece inalterada.

Em relação às colunas ausentes dos aeroportos:

- Em `aero_tcp` é o valor do destino.
- Em `aero_tcr` é o valor do destino.

In [97]:
merge1.head(2).T

,0,1
flightid,2127312fea57d093815e49b5aefee46d,e20845be25ed24b87131066d865cd9bc
origem,SBBR,SBFL
destino,SBGL,SBSP
dt_dep,2022-06-01 08:08:46,2022-06-01 09:12:29
dt_arr,2022-06-01 09:27:43,2022-06-01 10:01:07
hora_ref,2022-06-01 08:00:00,2022-06-01 09:00:00
esperas,0,0
hora_esperas,2022-06-01 07:00:00,2022-06-01 08:00:00
aero_esperas,SBGL,SBSP
hora_metaf,2022-06-01 09:00:00,2022-06-01 10:00:00


In [ ]:
merge1.to_csv('idsc_train.csv', index=False)

### Exemplo de como usar Metar

In [237]:
from metar import Metar
obs = Metar.Metar('METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T02270215')
print (obs.string())

station: KEWR
type: routine report, cycle 19 (automatic report)
time: Mon Sep 11 18:51:00 2023
temperature: 22.7 C
dew point: 21.5 C
wind: variable at 3 knots, gusting to 19 knots
peak wind: WNW at 28 knots at 18:17
wind shift: 18:12
visibility: 2 miles
visual range: on runway 04R, from 3000 to greater than 6000 feet
pressure: 1011.5 mb
weather: thunderstorm with rain; mist
sky: a few clouds at 1500 feet
     broken cumulonimbus at 4000 feet
     broken clouds at 6500 feet
     overcast at 20000 feet
sea-level pressure: 1011.4 mb
1-hour precipitation: 0.13in
remarks:
- Automated station (type 2)
- peak wind 28kt from 290 degrees at 18:17
- wind shift at 18:12
- frequent lightning (intracloud,cloud-to-cloud,cloud-to-ground)
- thunderstorm overhead and NW
- TSB05RAB22 -N-E MOV NE
METAR: METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T